In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Multi-fidelity (MF) optimization

In most cases it is better to do many cheap evaluations of an approximation to the target function than it is to only optimize the target function. This example demonstrates the 'multi-fidelity' capabilities of xopt. 

We follow the implementation of multi-fidelity bayesian optimization used in botorch https://botorch.org/tutorials/multi_fidelity_bo to optimize the synthetic test function AugmentedHartmann https://botorch.org/api/test_functions.html.

The difference between normal Bayesian optimization and MF optimization is that we specify a 'cost' to making observations at a given fidelity. For this example we assume a base cost of 5 and a fidelity cost between 0-1. The algorithm should make many observations at lower fidelity relative to higher fidelity, lowering the total observation cost. 

NOTE: The cost parameter is required to be the LAST element of the variables list. Also this method is best suited for parallel observations of the test function.

In [2]:
# Import the class
from xopt import Xopt
from xopt.bayesian.generators.multi_fidelity import MultiFidelityGenerator
from xopt.bayesian.models.models import create_multi_fidelity_model
from botorch.test_functions.multi_fidelity import AugmentedHartmann


The `Xopt` object can be instantiated from a JSON or YAML file, or a dict, with the proper structure.

Here we will make one

In [3]:
import yaml
# Make a proper input file. 
YAML = """
xopt: {output_path: null, verbose: true}

algorithm:
  name: multi_fidelity
  options:  
      n_initial_samples: 16
      n_steps: 10
      verbose: True
      use_gpu: False
      generator_options:                     ## options for bayesian exploration acquisition function
          batch_size: 4                      ## batch size for parallelized optimization


simulation: 
  name: test_multi_fidelity
  evaluate: xopt.evaluators.test_multi_fidelity.evaluate

vocs:
  name: test_multi_fidelity
  description: null
  simulation: test_multi_fidelity
  templates: null
  variables:
    x1: [0, 1.0]
    x2: [0, 1.0]
    x3: [0, 1.0]
    x4: [0, 1.0]
    x5: [0, 1.0]
    x6: [0, 1.0]
    cost: [0, 1.0]                          ## NOTE: THIS IS REQUIRED FOR MULTI-FIDELITY OPTIMIZATION
  objectives:
    y1: 'MINIMIZE'
  linked_variables: {}
  constants: {a: dummy_constant}

"""
config = yaml.safe_load(YAML)

In [4]:
X = Xopt(config)
X

Loading config as dict.



            Xopt 
________________________________           
Version: 0.4.3+91.g62835c4.dirty
Configured: True
Config as YAML:
xopt: {output_path: null, verbose: true, algorithm: cnsga}
algorithm:
  name: multi_fidelity
  function: xopt.bayesian.algorithms.multi_fidelity_optimize
  options:
    n_initial_samples: 16
    n_steps: 10
    verbose: true
    use_gpu: false
    generator_options: {batch_size: 4}
    custom_model: !!python/name:xopt.bayesian.models.models.create_multi_fidelity_model ''
    restart_file: null
    initial_x: null
simulation:
  name: test_multi_fidelity
  evaluate: xopt.evaluators.test_multi_fidelity.evaluate
  options: {extra_option: abc}
vocs:
  name: test_multi_fidelity
  description: null
  simulation: test_multi_fidelity
  templates: null
  variables:
    x1: [0, 1.0]
    x2: [0, 1.0]
    x3: [0, 1.0]
    x4: [0, 1.0]
    x5: [0, 1.0]
    x6: [0, 1.0]
    cost: [0, 1.0]
  objectives: {y1: MINIMIZE}
  linked_variables: {}
  constants: {a: dummy_constant}
 

# Run BayesOpt

In [5]:
# Pick one of these
from concurrent.futures import ThreadPoolExecutor as PoolExecutor
#from concurrent.futures import ProcessPoolExecutor as PoolExecutor

executor = PoolExecutor()
# This will also work. 
#executor=None

In [6]:
# Change max generations
X.run(executor=executor)
results = X.results

Starting at time 2021-08-23T10:16:10-05:00
started running optimization with generator: <xopt.bayesian.generators.multi_fidelity.MultiFidelityGenerator object at 0x000001D7A3E458B0>
submitting initial candidates at time 2021-08-23T10:16:10-05:00
starting optimization loop
Model creation time: 0.1712 s
Candidate generation time: 23.54 s
Candidate(s): tensor([[0.1939, 0.0187, 0.5065, 0.4093, 0.3505, 0.9215, 0.0000],
        [0.2378, 0.0745, 0.4097, 0.4640, 0.2229, 0.7369, 0.0000],
        [0.1095, 0.1947, 0.3469, 0.3761, 0.3078, 0.7746, 0.0000],
        [0.2646, 0.1889, 0.3177, 0.3946, 0.2463, 0.9560, 0.0000]],
       dtype=torch.float64)
submitting candidates at time 2021-08-23T10:16:34-05:00
Model creation time: 0.163 s
Candidate generation time: 24.94 s
Candidate(s): tensor([[9.1036e-02, 3.5225e-01, 3.0867e-01, 3.7316e-01, 2.5653e-01, 7.1178e-01,
         0.0000e+00],
        [6.7829e-02, 2.1020e-01, 4.0762e-01, 3.8823e-01, 4.3017e-01, 7.0235e-01,
         0.0000e+00],
        [1.0431

### Get highest fidelity global optimum

In [7]:
# create generator object
gen = MultiFidelityGenerator()

[autoreload of xopt.bayesian.generators.multi_fidelity failed: Traceback (most recent call last):
  File "C:\ProgramData\Miniconda3\envs\xopt\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\ProgramData\Miniconda3\envs\xopt\lib\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "C:\ProgramData\Miniconda3\envs\xopt\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "C:\ProgramData\Miniconda3\envs\xopt\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 846, in exec_module
  File "<frozen importlib._bootstrap_external>", line 983, in get_code
  File "<frozen importlib._bootstrap_external>", line 913, in source_to_code
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames

In [8]:
# create model
model = create_multi_fidelity_model(results['variables'], results['corrected_objectives'], results['corrected_constraints'], X.vocs)

In [9]:
## NOTE: we want to get the minimum evaluated at the highest fidelity -> make sure to use get_recommendation
rec = gen.get_recommendation(model, X.vocs)
problem = AugmentedHartmann(negate=False)
problem(rec) ## NOTE: the correct global minimum is -3.32237

tensor([-3.3217])